In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, applications
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import qiskit
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import os
import random

2024-05-07 17:11:04.095188: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 17:11:04.096870: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 17:11:04.129192: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 17:11:04.130020: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 17:11:04.721234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
# Ensure Reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Hardware Setup
print(tf.config.list_physical_devices())
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2024-05-07 17:11:06.481261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 17:11:06.481636: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Dataset Loading Functions
def load_plantdoc():
    import pathlib
    from tensorflow.keras.utils import image_dataset_from_directory

    data_dir = pathlib.Path('data/train')
    train_dataset = image_dataset_from_directory(data_dir, shuffle=True, image_size=(256, 256), batch_size=32)
    
    data_dir = pathlib.Path('data/test')
    test_dataset = image_dataset_from_directory(data_dir, shuffle=True, image_size=(256, 256), batch_size=32)

    return train_dataset, test_dataset

def load_plantvillage():
    (train_dataset, test_dataset), ds_info = tfds.load('plant_village', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
    
    def format_image(image, label):
        image = tf.image.resize(image, (256, 256)) / 255.0
        return image, label

    train_dataset = train_dataset.map(format_image).batch(32).shuffle(1000)
    test_dataset = test_dataset.map(format_image).batch(32)

    return train_dataset, test_dataset

def load_datasets():
    train_plantdoc, test_plantdoc = load_plantdoc()
    #train_plantvillage, test_plantvillage = load_plantvillage()
    return (train_plantdoc, test_plantdoc)#, (train_plantvillage, test_plantvillage)


In [4]:
# CNN Models
def simple_cnn():
    model = models.Sequential([
        layers.InputLayer(input_shape=(256, 256, 3)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

def resnet_50():
    base_model = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

def vgg16():
    base_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

def densenet121():
    base_model = applications.DenseNet121(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

# QCNN
def qcnn_model():
    # Quantum Circuit Definition
    n_qubits = 4
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit(inputs, weights):
        qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    weight_shapes = {"weights": (2, n_qubits, 3)}
    qlayer = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=n_qubits)

    model = models.Sequential([
        layers.Input(shape=(256, 256, 3)),
        layers.Flatten(),
        layers.Dense(n_qubits),
        qlayer,
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model


In [5]:
# Hyperparameter Optimization Function
def optimize_hyperparameters(model_fn, train_data, test_data, model_name):
    best_score = 0
    best_params = {}
    
    for batch_size in [32, 64]:
        for learning_rate in [0.001, 0.0001]:
            for dropout_rate in [0.2, 0.5]:
                model = model_fn()
                model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])
                history = model.fit(train_data, epochs=10, validation_data=test_data, batch_size=batch_size)
                y_true = []
                y_pred = []

                for images, labels in test_data:
                    preds = model.predict(images)
                    y_pred.extend(tf.argmax(preds, axis=1))
                    y_true.extend(labels)

                accuracy = accuracy_score(y_true, y_pred)
                precision = precision_score(y_true, y_pred, average='weighted')
                recall = recall_score(y_true, y_pred, average='weighted')
                f1 = f1_score(y_true, y_pred, average='weighted')

                if accuracy > best_score:
                    best_score = accuracy
                    best_params = {
                        "batch_size": batch_size,
                        "learning_rate": learning_rate,
                        "dropout_rate": dropout_rate,
                        "accuracy": accuracy,
                        "precision": precision,
                        "recall": recall,
                        "f1": f1
                    }

    print(f"Best Parameters for {model_name}: {best_params}")
    return best_params


In [6]:
# Main Evaluation Function
def evaluate_models():
    datasets = load_datasets()
    train_plantdoc, test_plantdoc = datasets[0]
    train_plantvillage, test_plantvillage = datasets[1]

    models_to_test = {
        "Simple CNN": simple_cnn,
        "ResNet-50": resnet_50,
        "VGG16": vgg16,
        "DenseNet121": densenet121,
        "QCNN": qcnn_model
    }

    results = {}
    for model_name, model_fn in models_to_test.items():
        results[model_name] = {
            "PlantDoc": optimize_hyperparameters(model_fn, train_plantdoc, test_plantdoc, model_name),
            "PlantVillage": optimize_hyperparameters(model_fn, train_plantvillage, test_plantvillage, model_name)
        }

    print("\n\nFinal Results:")
    for model_name, dataset_results in results.items():
        print(f"\n{model_name}:")
        for dataset, result in dataset_results.items():
            print(f"{dataset}: {result}")

evaluate_models()


Found 2342 files belonging to 28 classes.
Found 236 files belonging to 27 classes.


2024-05-07 17:11:06.832863: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2342]
	 [[{{node Placeholder/_4}}]]
2024-05-07 17:11:06.833407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2342]
	 [[{{node Placeholder/_4}}]]


ValueError: too many values to unpack (expected 2)